<a href="https://colab.research.google.com/github/Tarun2733/game-recommendation-system/blob/main/gameModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Pandas and Numpy

In [2]:
import pandas as pd
import numpy as np

In [3]:
games = pd.read_csv('/content/steam.csv')
tags_df = pd.read_csv('/content/steamspy_tag_data.csv')

